# Prova NeoSemantics

https://towardsdatascience.com/traveling-tourist-part-1-import-wikidata-to-neo4j-with-neosemantics-library-f80235f40dc5

In [5]:
from graphdatascience import GraphDataScience
from neo4j import GraphDatabase



In [6]:

driver = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j','simone98'))
session = driver.session()


In [7]:
q1='''CALL n10s.graphconfig.init({handleVocabUris: "IGNORE"})'''# Inserire vincolo
#session.run(q1)

In [ ]:
q2='''CALL apoc.schema.assert(
  {State:['id'], City:['id']},
  {Resource:['uri']})'''# Richiamare schema
#session.run(q2)

In [8]:
q3='''WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT{ ?item a sch:Monument; 
            sch:name ?monumentName; 
            sch:location ?location; 
            sch:img ?imageAsStr; 
            sch:ARCHITECTURE ?architecture. 
           ?architecture a sch:Architecture; 
            sch:name ?architectureName } 
WHERE { ?item wdt:P31 wd:Q4989906 . 
        ?item wdt:P17 wd:Q29 . 
        ?item rdfs:label ?monumentName . 
         filter(lang(?monumentName) = "en") 
        ?item wdt:P625 ?location . 
        ?item wdt:P149 ?architecture . 
        ?architecture rdfs:label ?architectureName .
         filter(lang(?architectureName) = "en") 
        ?item wdt:P18 ?image . 
         bind(str(?image) as ?imageAsStr) }' AS sparql 
CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" +   
   apoc.text.urlencode(sparql),"JSON-LD", 
   { headerParams: { Accept: "application/ld+json"} , 
     handleVocabUris: "IGNORE"}) 
YIELD terminationStatus, triplesLoaded
RETURN terminationStatus, triplesLoaded''' #Caricare RDF  da WikiData
#session.run(q3)

In [11]:
host = "bolt://localhost:7687"
user = "neo4j"
password= "simone98"


gds = GraphDataScience(host, auth=(user, password))

print(gds.version())

2.4.6


In [ ]:
#Con questa query scarico dati da wikidata su causa e conseguenza delle malattie
Q = '''WITH 'PREFIX sch: <http://schema.org/> 
CONSTRUCT {
  ?item a sch:MedicalCondition;
        sch:name ?diseaseName;
        sch:cause ?cause;
        sch:consequence ?consequence.
  ?cause a sch:MedicalCondition;
        sch:name ?causeName.
  ?consequence a sch:MedicalCondition;
              sch:name ?consequenceName.
}
WHERE {
  ?item wdt:P31 wd:Q12136;  # Malattia (Q12136)
        rdfs:label ?diseaseName.
  FILTER(LANG(?diseaseName) = "en").
  
  OPTIONAL { 
    ?item wdt:P828 ?causeItem. # Proprietà per la causa della malattia
    ?causeItem rdfs:label ?causeLabel.
    FILTER(LANG(?causeLabel) = "en").
    BIND(sch:cause AS ?cause).
  }

  OPTIONAL { 
    ?item wdt:P780 ?consequenceItem. # Proprietà per la conseguenza della malattia
    ?consequenceItem rdfs:label ?consequenceLabel.
    FILTER(LANG(?consequenceLabel) = "en").
    BIND(sch:consequence AS ?consequence).
  }
}' AS sparql 

CALL n10s.rdf.import.fetch(
  "https://query.wikidata.org/sparql?query=" +   
   apoc.text.urlencode(sparql),
  "JSON-LD", 
  { headerParams: { Accept: "application/ld+json"} , 
    handleVocabUris: "IGNORE"
  }) 

YIELD terminationStatus, triplesLoaded
RETURN terminationStatus, triplesLoaded;'''

Si possono scaricare i grafi da WikiData attraverso query SPARQL il problema è che i dat provengono da Wikipeida quindi come fonte non è affidabile in ambito medico